In [ ]:
pip install -q yahoo_fin

In [2]:
!mkdir -p data results logs


In [3]:
from google.colab import drive
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [ ]:
pip install  git+https://github.com/ahsank/runml@testv2.01#egg=runml

In [5]:
import runml.pipeline
from runml import pipeline,findata


In [6]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    #df['Gain_f'] = (2*df['Gain']+df['Gain_l']+df['Gain_h'])/4
    df['Gain_f'] = df[['Gain','Gain_l']].max(axis=1)
    df['Gain_f'] = df[['Gain_h', 'Gain_f']].min(axis=1)
    df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain_f'])
    df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
    return df

In [7]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
new = []
model = "gstock-10b-test"
tickers = [ 'AEXAF', 'ALSMY', 'AMSSY', 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY', 'CAAP', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX',
            'DBOEY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY', 'EQNR', 'ERIC', 'ERJ','ESEA',
            'FANUY', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'OZON', 'RACE', 'LYB', 'LVMUY',
            'MYTE', 'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'VWAPY', 'YARIY', 'ZURVY']


In [ ]:
findata.EPOCHS=200
model = "val-10b-test"
new = ['CCI', 'TMUS', 'CLS', 'POWL', 'ANF', 'MOD', 'STRL',  'MPC', 'BXC',
       'URI', 'DINO', 'TEX', 'POR', 'HIW']
tickers = ['AAL', 'ALB', 'ANF', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BGFV', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'EMN',
            'F', 'FCX', 'FIS', 'FLO', 'FMS', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN', 'KD', 'KHC', 'KMB',
             'M', 'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL', 'PXD', 'RCL', 'RH',
            'SAFE', 'SCCO', 'SPTN', 'SPG', 'STRL', 'SWK',
            'T', 'TCN', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']

In [ ]:
findata.EPOCHS=200
model = "cry-11a-test"
tickers = [ 'ADA-USD', 'BTC-USD', 'DOT-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'LINK-USD', 'LTC-USD', 'MATIC-USD', 'SOL-USD', 'XRP-USD']

In [ ]:
findata.EPOCHS=200
model = "etf-11a-test"
tickers = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]

In [ ]:
findata.EPOCHS=200
model = "mcap-11b-test"
tickers = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX', 'TMO', 'TSLA', 'TXN', 'UNH', 'UPS']

In [ ]:
pipeline.IS_VERBOSE = False
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


df  = pipeline.runModelCombinedVola(tickers, model, mod, False, loss=lossfn)


In [9]:
addAllocHL(df, 0.10, 1)
df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
#df.sort_values('Alloc', ascending=False).head(50)


,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
63,THQQF,2.17,0.73,123.02,128.82,2.20,3.280000,0.49,0.55,111.94,94.07,4.650000,1.12,0.62,46.12,98.32,2.230000,0.02,0.49,6.748980
57,SE,41.07,0.87,1183.67,1621.30,37.50,54.560001,0.46,0.70,1462.42,1364.54,52.119999,0.39,0.72,1294.31,1388.47,37.299999,-0.01,0.39,8.366667
25,FQVLF,4.21,0.88,225.00,161.85,10.04,12.100000,0.21,0.70,148.72,162.78,17.590000,0.75,0.67,159.24,181.77,11.000000,0.10,0.21,8.228571
1,ALSMY,1.27,0.88,20.19,30.18,1.31,1.550000,0.18,0.59,9.05,14.32,1.800000,0.37,0.77,16.27,26.49,1.280000,-0.02,0.18,8.133333
2,AMSSY,2.07,0.79,41.39,93.39,1.72,1.600000,-0.07,0.64,37.78,68.84,2.030000,0.18,0.70,42.97,82.11,2.090000,0.22,0.18,6.733333
10,COTY,2.85,0.90,70.27,80.92,11.04,9.240000,-0.16,0.71,68.39,49.00,12.150000,0.10,0.66,57.00,38.76,9.600000,-0.13,-0.13,8.230769
23,ESEA,2.18,0.85,192.59,150.98,28.19,24.150000,-0.14,0.76,232.24,144.79,28.790001,0.02,0.67,186.88,145.52,24.030001,-0.15,-0.14,7.428571
34,MELI,476.71,0.93,12901.33,8892.14,1525.00,1356.680054,-0.11,0.73,11898.59,5362.59,1620.650024,0.06,0.66,9113.81,8903.42,1345.630005,-0.12,-0.11,8.663636
49,POAHF,34.69,0.88,415.60,488.13,49.08,57.040001,0.16,0.72,317.24,409.56,55.180000,0.12,0.72,438.79,489.02,47.590000,-0.03,0.12,7.800000
62,TGLS,3.06,0.85,252.81,198.18,34.45,24.930000,-0.28,0.75,278.07,161.42,33.570000,-0.03,0.70,222.77,198.69,30.410000,-0.12,-0.12,7.250000


In [ ]:
findata.EPOCHS=200
model = "vols-10a-test"
pending = ['ASTR']
tickers = [ 'AIRI', 'AMSC', 'APRN', 'ACHR', 'AUGX', 'AUR',   'ATOM',
            'BKKT', 'BGSF', 'BLNK', 'BSGM',
            'CARS', 'CLOV', 'CZOO',
            'DLO',  'DM', 'DMTK', 'DPSI', 'EVGO', 'EXPR', 'FSLY', 'FSR', 'FTCH',
            'GOVX', 'GRAB', 'GTE', 'HIMS', 'HIVE',
            'INLX',  'INVZ', 'JOBY',  'KIND', 'KULR',
            'LAZR', 'LUMN', 'NNOX', 'MTTR',
            'NEPH', 'NSTG', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PTON', 'RAD', 'REAL', 'REI', 'RIG', 'RKLB', 'RVYL',
            'SFIX', 'SOUN', 'STEM', 'SFT',  'TELL', 'VTEX', 'YEXT']
future = ['WKME', 'CXM']

In [ ]:
pending = ['PAR']
findata.EPOCHS=200
model = "ipos-10a-test"
new = ['SMCI', 'TRIP']
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRCT', 'CRWD',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DXCM',
           'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FOUR', 'GFS', 'GTLB', 'GLBE', 'GOGO',
           'INDI', 'INMD', 'INTA', 'IOT', 'IRDM', 'JKS', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
           'MTCH',  'NVCR','OKTA', 'OLED', 'OSTK',
           'PERI', 'PANW', 'PAYO', 'PAYX', 'PD', 'PLUG', 'PI', 'PINS', 'PUBM',
           'RBLX', 'SNAP', 'SNOW', 'SPLK', 'SQ', 'SPOT',
           'TEAM', 'TDOC', 'TOST', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']

In [11]:
findata.EPOCHS=200
model = "cetf-11b-test"
tickers = ['ARGT', 'EWD', 'FLN', 'FM',
            'ECH', 'EWA', 'EWI', 'EWC', 'EWM', 'EWP', 'EWW', 'EWY', 'EWZ',
            'IDX', 'ILF', 'KEMQ', 'THD', 'TUR', 'XCEM']

In [ ]:
findata.EPOCHS=200
model = "com-11a-test"
tickers = [ 'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'UUP', 'VTIP', 'WEAT']

In [ ]:
lossfn = "huber_loss"
pipeline.IS_VERBOSE = False
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='low', trading=pipeline.LowTrading)[0]
#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='high', trading=pipeline.HighTrading)[0]
df  = pipeline.runModelCombinedVola(tickers, model, mod, True, loss=lossfn)

In [13]:
addAllocHL(df, 0.10, 1)
df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
#df.sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
9,EWP,16.64,0.91,95.76,103.15,29.89,27.290001,-0.09,0.68,74.20,40.55,29.330000,-0.02,0.68,95.30,76.87,27.530001,-0.08,-0.08,7.975000
15,KEMQ,12.02,0.80,95.21,112.36,14.82,16.270000,0.10,0.55,10.16,59.98,16.520000,0.11,0.74,93.85,115.06,14.650000,-0.01,0.10,6.000000
6,EWI,16.50,0.94,120.64,130.18,32.39,30.410000,-0.06,0.67,72.38,51.84,33.169998,0.02,0.70,107.40,82.48,30.440001,-0.06,-0.06,8.400000
11,EWY,38.48,0.93,269.24,299.09,62.91,66.699997,0.06,0.66,240.81,150.51,66.129997,0.05,0.73,250.34,280.54,61.349998,-0.02,0.05,7.900000
3,FM,18.13,0.93,81.65,91.30,26.88,27.920000,0.04,0.65,19.52,37.31,27.940001,0.04,0.68,48.77,93.40,25.540001,-0.05,0.04,7.550000
16,THD,45.40,0.93,271.38,329.50,62.75,65.339996,0.04,0.71,201.94,160.18,65.230003,0.04,0.69,184.92,208.10,61.970001,-0.01,0.04,7.550000
8,EWM,17.00,0.93,59.43,89.52,21.23,22.120001,0.04,0.65,36.75,45.65,22.680000,0.07,0.73,51.67,58.77,21.190001,-0.00,0.04,7.550000
13,IDX,10.12,0.86,53.11,71.75,16.68,17.450001,0.05,0.76,66.18,47.60,18.400000,0.10,0.71,61.33,62.37,16.469999,-0.01,0.05,5.800000
4,ECH,15.35,0.91,150.56,126.19,26.62,25.510000,-0.04,0.77,154.57,102.46,27.540001,0.03,0.84,149.91,188.49,24.670000,-0.07,-0.04,6.850000
14,ILF,12.67,0.91,127.80,117.11,27.75,26.969999,-0.03,0.70,101.96,64.81,27.629999,-0.00,0.79,143.37,150.13,25.750000,-0.07,-0.03,6.100000


In [ ]:
pip install seaborn

In [ ]:
import seaborn as sns

Todos:



1.   Load previously saved model before training
2.   Allow eval only without test/train spliting
3.   Incremental data load
4.    Add S&P, QQQ etc to the model
5. Save model to google drive






In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !cp  /content/drive/MyDrive/colab/results/* ./results/
else:
  ! cp ~/Google\ Drive/My\ Drive/colab/results/* ./results/

Run following in local Jupyter

Run following 2 cells for colab to import libraries

Run following if the library code is modified locally

In [ ]:
!git clone https://github.com/ahsank/runml

In [ ]:
import os
import sys
path = os.path.expanduser('~/src/runml')
print(path)
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata

/Users/ahsank/src/runml


In [ ]:
import importlib
import sys
path = 'runml'
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata


In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
from importlib import reload
reload(findata)
reload(pipeline)

<module 'runml.pipeline' from '/Users/ahsank/src/runml/runml/pipeline.py'>

In [ ]:
import pandas as pd
df1 = pipeline.fetch_data('ABNB')
#df1.sort_index(inplace=True)
df1 = df1.round(3)

df2 = pd.read_csv('data/ABNB_2023-04-15.csv', index_col=0)
df2.index =  pd.to_datetime(df2.index)
# df2 = df2.round(3)
#data2 = data.tail(1000)
#pd.concat([df1,df2]).drop_duplicates(keep=False)
#df1.compare(df2)
#df2.index.dtype
df1

In [ ]:
display(df.sort_values('Gain', ascending=False))
display(df[["Buy", "Sell", "Total"]].sum()/len(df))


In [ ]:
results['ABNB'].pdata.__dict__.keys()

In [ ]:
findata.EPOCHS=10
tickers1 = ['ARKK', 'ARKW', 'DAPP', 'DTEC', 'EEM', 'FPX',
            'GBTC', 'GLD',
            'ICLN', 'IJR', 'IPO', 'IPOS', 'IWM',
            'JETS', 'KEMQ',
            'MGK', 'MGV', 'MOAT', 'MTUM',
            'QQQ', 'SLV','SMH', 'SMOG',
            'SPY', 'VNQ', 'VT', 'VTI', 'WDIV',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME' ]
mod = pipeline.RateReturnOnly(pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap()]))
df1 = pipeline.runModelCombined(tickers1, 'etf2b', mod, True)

In [ ]:
df1.sort_values('Gain', ascending=False)

In [ ]:
findata.EPOCHS=200
tickers3 = [
           'AHT', 'AMSC', 'ASTR', 'ATOM',
           'BKKT', 'BGFV', 'BGSF', 'CBNT', 'CLOV',
           'DNMR', 'ERJ', 'EVGO',
           'FSLY',  'FTCH', 'GOGO', 'HIVE',
           'ILAL', 'INLX', 'JMIA', 'JOBY',  'KULR', 'MTTR',
           'MYTE', 'NEPH', 'ONDS', 'MQ',
           'PETS', 'PTON', 'SFIX', 'SFT', 'STNE',
           'ULH', 'VRAR', 'WISH']
lossfn = "huber_loss"
# lossfn = "mean_squared_error"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df3, results = pipeline.runModelCombined(tickers3, 'vols', mod, False, loss=lossfn)

In [ ]:
df3

In [ ]:
import pandas as pd
seq = list()
for ticker, result in results.items():
    tempdf = result.final_df
    tempdf['predicted_rate'] = (tempdf['adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    tempdf['true_rate'] = (tempdf['true_adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    seq.append(tempdf[['ticker', 'date', "predicted_rate", 'true_rate']])

detailstat = pd.concat(seq)



In [ ]:
normalstat = detailstat[detailstat['predicted_rate'].between(-2,2) & detailstat['true_rate'].between(-2,2)]
largestat = normalstat[~ normalstat['predicted_rate'].between(-0.25,0.25) & ~ normalstat['true_rate'].between(-0.25,0.25)]
largestat.sample(10)

In [ ]:
ax = largestat.plot.scatter(x='predicted_rate', y='true_rate',c='DarkBlue', figsize=(10,10))
# ax.set_yscale('log')
# ax.set_xscale('log')

In [ ]:
# import matplotlib.pyplot as plt

# plt.hist([normalstat['predicted_rate'],normalstat['predicted_rate']],
#          bins=100, range=(-1,1), color = ['r','g'])

corr = largestat[["predicted_rate", "true_rate"]].corr()

corr.style.background_gradient(cmap='coolwarm')

In [ ]:

sns.regplot(x=largestat["predicted_rate"], y=largestat["true_rate"])


In [ ]:
sns.heatmap(normalstat[["predicted_rate", "true_rate"]].corr(), annot = True, fmt='.2g',cmap= 'coolwarm')

In [14]:
!rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/


sending incremental file list
cetf-11b-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
cetf-11b-test-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
cetf-11b-test-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 9,647,436 bytes  received 73 bytes  19,295,018.00 bytes/sec
total size is 176,683,760  speedup is 18.31
